In [93]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
import pickle

import numpyro
import numpyro.handlers as handlers
import jax.numpy as jnp
import jax

from jax import random
import optax
from numpyro.infer import Predictive, SVI, Trace_ELBO

from src.models.CCVAE import CCVAE
from src.models.encoder_decoder import MNISTEncoder, MNISTDecoder, CIFAR10Encoder, CIFAR10Decoder
from src.data_loading.loaders import get_data_loaders

import matplotlib.pyplot as plt

In [95]:
# Set up random seed
seed = 42

# DATASET
dataset_name = "MNIST" # use "CIFAR10"

encoder_class = MNISTEncoder if dataset_name=="MNIST" else CIFAR10Encoder
decoder_class = MNISTDecoder if dataset_name=="MNIST" else CIFAR10Decoder
distribution = "bernoulli" if dataset_name=="MNIST" else "laplace"

In [96]:
img_shape, loader_dict, size_dict = get_data_loaders(dataset_name=dataset_name, 
                                          p_test=0.2, 
                                          p_val=0.2, 
                                          p_supervised=0.05, 
                                          batch_size=10, 
                                          num_workers=0, 
                                          seed=seed)

scale_factor = 1.

Successfully loaded MNIST dataset.
Total num samples 60000
Num test samples: 12000
Num validation samples: 9600
Num supervised samples: 1920
Num unsupervised samples: 36480


In [97]:
ccvae = CCVAE(encoder_class, 
               decoder_class, 
               10, 
               50, 
               img_shape, 
               scale_factor=scale_factor, 
               distribution=distribution,
               multiclass=False
)

In [98]:
optimizer = optax.adam(1e-3)

In [99]:
svi_supervised = SVI(ccvae.model_supervised, 
            ccvae.guide_supervised, 
            optim=optimizer, 
            loss=Trace_ELBO()
)
svi_classify = SVI(ccvae.model_classify, 
            ccvae.guide_classify, 
            optim=optimizer, 
            loss=Trace_ELBO()
)

In [100]:
state = svi_supervised.init(
    random.PRNGKey(seed), 
    xs=jnp.ones((1,)+img_shape), 
    ys=jnp.ones((1), dtype=jnp.int32)
)
svi_classify.init(
    random.PRNGKey(seed), 
    xs=jnp.ones((1,)+img_shape), 
    ys=jnp.ones((1), dtype=jnp.int32)
)
print("done")

done


In [101]:
loader_supervised = loader_dict["supervised"]

In [102]:
for batch in loader_supervised:
    x, y = batch
    svi_supervised.update(state, xs=x, ys=y)
    svi_classify.update(state, xs=x, ys=y)
    break

In [103]:
svi_unsupervised = SVI(ccvae.model_unsupervised, 
            ccvae.guide_unsupervised, 
            optim=optimizer, 
            loss=Trace_ELBO()
)

In [104]:
state = svi_unsupervised.init(
    random.PRNGKey(seed), 
    xs=jnp.ones((1,)+img_shape)
)

C:\Users\Matthieu Dinot\AppData\Local\Temp\ipykernel_12000\784847552.py:1: UserWarning: Currently, SVI with Trace_ELBO loss does not support models with discrete latent variables
  state = svi_unsupervised.init(


In [105]:
loader_unsupervised = loader_dict["unsupervised"]

In [106]:
for batch in loader_unsupervised:
    x = batch
    svi_unsupervised.update(state, xs=x)
    break